# Chapter 32: Graph Patterns

> *"Graph problems may seem daunting, but they often follow recognizable patterns. Recognizing these patterns is the key to unlocking efficient solutions."* — Anonymous

---

## 32.1 Introduction to Graph Patterns

Many graph problems in coding interviews and competitions are variations of a few fundamental patterns. By recognizing these patterns, you can quickly identify the appropriate algorithm and data structures to solve the problem efficiently. This chapter categorizes common graph problem patterns and provides templates and insights for each.

### 32.1.1 Why Graph Patterns Matter

```
┌─────────────────────────────────────────────────────────────────────┐
│                    IMPORTANCE OF GRAPH PATTERNS                       │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  1. EFFICIENCY: Recognizing the pattern helps you choose the right  │
│     algorithm (BFS, DFS, Dijkstra, Union-Find) without reinventing  │
│     the wheel.                                                      │
│                                                                      │
│  2. PROBLEM-SOLVING SPEED: In interviews, time is critical.         │
│     Pattern recognition accelerates solution design.                │
│                                                                      │
│  3. VARIETY REDUCTION: Many seemingly different problems reduce to  │
│     a small set of classic patterns.                                │
│                                                                      │
│  4. FOUNDATION FOR ADVANCED TOPICS: Understanding these patterns    │
│     prepares you for more complex graph algorithms.                 │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 32.2 Island Problems (Connected Components in Grids)

Island problems typically involve a 2D grid where cells are land ('1') or water ('0'). The goal is to count islands, find the largest island, or compute perimeter. These are essentially connected component problems on a grid graph.

### 32.2.1 Number of Islands (LeetCode 200)

**Problem:** Given a 2D grid of '1's (land) and '0's (water), count the number of islands. An island is surrounded by water and formed by connecting adjacent lands horizontally or vertically.

**Pattern:** DFS or BFS to mark visited cells.

**DFS Solution:**

```python
def num_islands(grid):
    if not grid:
        return 0
    rows, cols = len(grid), len(grid[0])
    count = 0
    
    def dfs(r, c):
        if r < 0 or r >= rows or c < 0 or c >= cols or grid[r][c] != '1':
            return
        grid[r][c] = '0'  # mark as visited (sink the island)
        dfs(r+1, c)
        dfs(r-1, c)
        dfs(r, c+1)
        dfs(r, c-1)
    
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '1':
                count += 1
                dfs(r, c)
    return count
```

**BFS Solution:**

```python
from collections import deque

def num_islands_bfs(grid):
    if not grid:
        return 0
    rows, cols = len(grid), len(grid[0])
    count = 0
    directions = [(1,0), (-1,0), (0,1), (0,-1)]
    
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '1':
                count += 1
                q = deque([(r,c)])
                grid[r][c] = '0'
                while q:
                    x, y = q.popleft()
                    for dx, dy in directions:
                        nx, ny = x+dx, y+dy
                        if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] == '1':
                            grid[nx][ny] = '0'
                            q.append((nx, ny))
    return count
```

**Complexity:** O(rows × cols) time and space (for recursion stack or queue).

### 32.2.2 Max Area of Island (LeetCode 695)

**Problem:** Find the maximum area of an island (number of connected '1's).

**Pattern:** DFS returns size.

```python
def max_area_of_island(grid):
    rows, cols = len(grid), len(grid[0])
    max_area = 0
    
    def dfs(r, c):
        if r < 0 or r >= rows or c < 0 or c >= cols or grid[r][c] == 0:
            return 0
        grid[r][c] = 0  # mark visited
        area = 1
        area += dfs(r+1, c)
        area += dfs(r-1, c)
        area += dfs(r, c+1)
        area += dfs(r, c-1)
        return area
    
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 1:
                max_area = max(max_area, dfs(r, c))
    return max_area
```

### 32.2.3 Island Perimeter (LeetCode 463)

**Problem:** Calculate the perimeter of a single island.

**Pattern:** For each land cell, contribute 4 minus number of adjacent land cells.

```python
def island_perimeter(grid):
    rows, cols = len(grid), len(grid[0])
    perimeter = 0
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 1:
                perimeter += 4
                if r > 0 and grid[r-1][c] == 1:
                    perimeter -= 2  # shared edge counted twice
                if c > 0 and grid[r][c-1] == 1:
                    perimeter -= 2
    return perimeter
```

---

## 32.3 Shortest Path in Grid

When a grid has weighted or unweighted edges, we need shortest path algorithms. Common variants:

- **Unweighted grid:** BFS gives shortest path in number of steps.
- **Grid with obstacles and weights:** Dijkstra or 0-1 BFS (if weights 0/1).
- **Grid with movement costs:** A* with heuristic.

### 32.3.1 Shortest Path in Binary Matrix (LeetCode 1091)

**Problem:** In an n×n grid, find the shortest path from top-left to bottom-right using 8-directional movement, only through 0 cells.

**Pattern:** BFS.

```python
from collections import deque

def shortest_path_binary_matrix(grid):
    n = len(grid)
    if grid[0][0] == 1 or grid[n-1][n-1] == 1:
        return -1
    if n == 1:
        return 1
    directions = [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]
    q = deque([(0,0,1)])  # (r, c, distance)
    grid[0][0] = 1  # mark visited
    while q:
        r, c, dist = q.popleft()
        for dr, dc in directions:
            nr, nc = r+dr, c+dc
            if 0 <= nr < n and 0 <= nc < n and grid[nr][nc] == 0:
                if nr == n-1 and nc == n-1:
                    return dist + 1
                grid[nr][nc] = 1
                q.append((nr, nc, dist+1))
    return -1
```

### 32.3.2 Minimum Path Sum with Obstacles (LeetCode 64) – DP, but also can be Dijkstra if weights vary.

**Pattern:** If each cell has a cost, Dijkstra.

```python
import heapq

def min_path_sum_dijkstra(grid):
    rows, cols = len(grid), len(grid[0])
    dist = [[float('inf')] * cols for _ in range(rows)]
    dist[0][0] = grid[0][0]
    pq = [(grid[0][0], 0, 0)]  # (cost, r, c)
    directions = [(1,0), (0,1)]  # down and right
    while pq:
        cost, r, c = heapq.heappop(pq)
        if r == rows-1 and c == cols-1:
            return cost
        if cost > dist[r][c]:
            continue
        for dr, dc in directions:
            nr, nc = r+dr, c+dc
            if 0 <= nr < rows and 0 <= nc < cols:
                new_cost = cost + grid[nr][nc]
                if new_cost < dist[nr][nc]:
                    dist[nr][nc] = new_cost
                    heapq.heappush(pq, (new_cost, nr, nc))
    return -1
```

### 32.3.3 0-1 BFS for Grid with Obstacles That Can Be Removed

**Problem:** Shortest path in grid where you can eliminate at most k obstacles (LeetCode 1293). This is BFS with state (r, c, k).

```python
from collections import deque

def shortest_path_with_obstacle_elimination(grid, k):
    rows, cols = len(grid), len(grid[0])
    if rows == 1 and cols == 1:
        return 0
    # visited[r][c][k_remaining]
    visited = [[[False] * (k+1) for _ in range(cols)] for _ in range(rows)]
    q = deque([(0,0,k,0)])  # (r, c, k_remaining, steps)
    visited[0][0][k] = True
    directions = [(1,0), (-1,0), (0,1), (0,-1)]
    while q:
        r, c, rem, steps = q.popleft()
        for dr, dc in directions:
            nr, nc = r+dr, c+dc
            if 0 <= nr < rows and 0 <= nc < cols:
                new_rem = rem - grid[nr][nc]  # if obstacle, rem decreases
                if new_rem >= 0 and not visited[nr][nc][new_rem]:
                    if nr == rows-1 and nc == cols-1:
                        return steps + 1
                    visited[nr][nc][new_rem] = True
                    q.append((nr, nc, new_rem, steps+1))
    return -1
```

---

## 32.4 Topological Sort Patterns

Topological sorting of a Directed Acyclic Graph (DAG) is used for scheduling problems with dependencies.

### 32.4.1 Course Schedule (LeetCode 207)

**Problem:** Given numCourses and prerequisites pairs [a, b] meaning b → a (take b before a), determine if it's possible to finish all courses (i.e., graph has no cycle).

**Pattern:** Kahn's algorithm (BFS) or DFS cycle detection.

**Kahn's Algorithm (BFS):**

```python
from collections import deque, defaultdict

def can_finish(numCourses, prerequisites):
    graph = defaultdict(list)
    indegree = [0] * numCourses
    for a, b in prerequisites:
        graph[b].append(a)
        indegree[a] += 1
    q = deque([i for i in range(numCourses) if indegree[i] == 0])
    count = 0
    while q:
        u = q.popleft()
        count += 1
        for v in graph[u]:
            indegree[v] -= 1
            if indegree[v] == 0:
                q.append(v)
    return count == numCourses
```

**DFS Cycle Detection:**

```python
def can_finish_dfs(numCourses, prerequisites):
    graph = defaultdict(list)
    for a, b in prerequisites:
        graph[b].append(a)
    state = [0] * numCourses  # 0 unvisited, 1 visiting, 2 visited
    def has_cycle(u):
        state[u] = 1
        for v in graph[u]:
            if state[v] == 1:
                return True
            if state[v] == 0 and has_cycle(v):
                return True
        state[u] = 2
        return False
    for i in range(numCourses):
        if state[i] == 0:
            if has_cycle(i):
                return False
    return True
```

### 32.4.2 Course Schedule II (LeetCode 210) – Return the order.

**Pattern:** Kahn's algorithm; if count == numCourses, return order; else empty list.

### 32.4.3 Alien Dictionary (LeetCode 269)

**Problem:** Given a sorted dictionary of an alien language, find the order of characters.

**Pattern:** Build graph from adjacent words (first differing character), then topological sort.

```python
def alien_order(words):
    graph = {c: set() for word in words for c in word}
    indegree = {c: 0 for c in graph}
    # build graph
    for i in range(len(words)-1):
        w1, w2 = words[i], words[i+1]
        min_len = min(len(w1), len(w2))
        # if prefix mismatch, invalid
        if len(w1) > len(w2) and w1[:min_len] == w2[:min_len]:
            return ""
        for j in range(min_len):
            if w1[j] != w2[j]:
                if w2[j] not in graph[w1[j]]:
                    graph[w1[j]].add(w2[j])
                break
    # compute indegree
    for u in graph:
        for v in graph[u]:
            indegree[v] += 1
    # Kahn's
    q = deque([c for c in graph if indegree[c] == 0])
    result = []
    while q:
        u = q.popleft()
        result.append(u)
        for v in graph[u]:
            indegree[v] -= 1
            if indegree[v] == 0:
                q.append(v)
    if len(result) != len(graph):
        return ""
    return ''.join(result)
```

---

## 32.5 Union-Find for Dynamic Connectivity

Union-Find (Disjoint Set Union) is perfect for problems involving connectivity queries with edge additions or for finding connected components.

### 32.5.1 Number of Connected Components in an Undirected Graph (LeetCode 323 – Premium, but classic)

**Problem:** Given n nodes and edges, find number of connected components.

**Pattern:** Union-Find.

```python
def count_components(n, edges):
    parent = list(range(n))
    rank = [0] * n
    def find(x):
        if parent[x] != x:
            parent[x] = find(parent[x])
        return parent[x]
    def union(x, y):
        xr, yr = find(x), find(y)
        if xr == yr:
            return 0
        if rank[xr] < rank[yr]:
            parent[xr] = yr
        elif rank[xr] > rank[yr]:
            parent[yr] = xr
        else:
            parent[yr] = xr
            rank[xr] += 1
        return 1
    components = n
    for u, v in edges:
        components -= union(u, v)
    return components
```

### 32.5.2 Redundant Connection (LeetCode 684)

**Problem:** Find an edge that can be removed to make the graph a tree.

**Pattern:** Union-Find; if adding an edge connects already connected nodes, it's redundant.

```python
def find_redundant_connection(edges):
    parent = list(range(len(edges)+1))  # 1-indexed nodes
    rank = [0] * (len(edges)+1)
    def find(x):
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x
    def union(x, y):
        xr, yr = find(x), find(y)
        if xr == yr:
            return False
        if rank[xr] < rank[yr]:
            parent[xr] = yr
        elif rank[xr] > rank[yr]:
            parent[yr] = xr
        else:
            parent[yr] = xr
            rank[xr] += 1
        return True
    for u, v in edges:
        if not union(u, v):
            return [u, v]
    return []
```

### 32.5.3 Accounts Merge (LeetCode 721)

**Problem:** Given list of accounts where each account has a name and emails, merge accounts with common emails.

**Pattern:** Union-Find on emails, then group by root.

---

## 32.6 Flood Fill

Flood fill is essentially DFS or BFS on a grid starting from a point, changing color.

### 32.6.1 Flood Fill (LeetCode 733)

**Problem:** Change all connected pixels of same color to a new color.

**DFS Solution:**

```python
def flood_fill(image, sr, sc, newColor):
    rows, cols = len(image), len(image[0])
    orig_color = image[sr][sc]
    if orig_color == newColor:
        return image
    
    def dfs(r, c):
        if r < 0 or r >= rows or c < 0 or c >= cols or image[r][c] != orig_color:
            return
        image[r][c] = newColor
        dfs(r+1, c)
        dfs(r-1, c)
        dfs(r, c+1)
        dfs(r, c-1)
    
    dfs(sr, sc)
    return image
```

**BFS Solution also possible.**

---

## 32.7 Cycle Detection in Graphs

### 32.7.1 Cycle in Undirected Graph

**Pattern:** DFS with parent tracking.

```python
def has_cycle_undirected(graph):
    visited = [False] * len(graph)
    def dfs(u, parent):
        visited[u] = True
        for v in graph[u]:
            if not visited[v]:
                if dfs(v, u):
                    return True
            elif v != parent:
                return True
        return False
    for i in range(len(graph)):
        if not visited[i]:
            if dfs(i, -1):
                return True
    return False
```

### 32.7.2 Cycle in Directed Graph

**Pattern:** Three-color DFS (or recursion stack) as shown in topological sort section.

---

## 32.8 Bipartite Graph Check

A graph is bipartite if its vertices can be colored with two colors such that no adjacent vertices share the same color.

**Pattern:** BFS/DFS 2-coloring.

```python
def is_bipartite(graph):
    color = {}  # 0 or 1
    def bfs(start):
        queue = [start]
        color[start] = 0
        while queue:
            u = queue.pop(0)
            for v in graph[u]:
                if v not in color:
                    color[v] = 1 - color[u]
                    queue.append(v)
                elif color[v] == color[u]:
                    return False
        return True
    for i in range(len(graph)):
        if i not in color:
            if not bfs(i):
                return False
    return True
```

---

## 32.9 Summary

```
┌─────────────────────────────────────────────────────────────────────┐
│                    GRAPH PATTERNS SUMMARY                            │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Island Problems (Grid Connectivity): DFS/BFS                       │
│  Shortest Path in Grid: BFS (unweighted), Dijkstra (weighted)       │
│  Topological Sort: Kahn's BFS, DFS with postorder                   │
│  Dynamic Connectivity: Union-Find                                    │
│  Flood Fill: DFS/BFS on grid                                         │
│  Cycle Detection: DFS with parent (undirected), 3-state DFS (directed)│
│  Bipartite Check: BFS/DFS 2-coloring                                │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 32.10 Practice Problems

### Island Problems
1. **Number of Islands** (LeetCode 200)
2. **Max Area of Island** (LeetCode 695)
3. **Island Perimeter** (LeetCode 463)
4. **Surrounded Regions** (LeetCode 130)
5. **Number of Closed Islands** (LeetCode 1254)

### Shortest Path in Grid
6. **Shortest Path in Binary Matrix** (LeetCode 1091)
7. **Minimum Path Sum** (LeetCode 64) – DP, but can be done with Dijkstra.
8. **The Maze** (LeetCode 490) – premium.
9. **The Maze II** (LeetCode 505) – premium (Dijkstra).
10. **Shortest Path to Get All Keys** (LeetCode 864) – BFS with state.

### Topological Sort
11. **Course Schedule** (LeetCode 207)
12. **Course Schedule II** (LeetCode 210)
13. **Alien Dictionary** (LeetCode 269)
14. **Sequence Reconstruction** (LeetCode 444) – premium.

### Union-Find
15. **Number of Connected Components in an Undirected Graph** (LeetCode 323) – premium.
16. **Redundant Connection** (LeetCode 684)
17. **Accounts Merge** (LeetCode 721)
18. **Regions Cut By Slashes** (LeetCode 959)

### Flood Fill
19. **Flood Fill** (LeetCode 733)

### Cycle Detection
20. **Graph Valid Tree** (LeetCode 261) – premium.
21. **Detect Cycles in 2D Grid** (LeetCode 1559)

### Bipartite
22. **Is Graph Bipartite?** (LeetCode 785)
23. **Possible Bipartition** (LeetCode 886)

---

## 32.11 Further Reading

1. **"Introduction to Algorithms" (CLRS)** – Graph algorithm chapters.
2. **"Algorithms"** by Robert Sedgewick – Graph algorithms.
3. **"Cracking the Coding Interview"** – Graph problems.
4. **LeetCode Explore – Graph** section.

---

> **Coming in Chapter 33**: **Modified Binary Search** – We'll explore binary search variations and parametric search.

---

**End of Chapter 32**